In [1]:
from select import select
from odps import ODPS
from odps import options
from odps.df import DataFrame

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

In [2]:
ACCESS_ID = 'LTAI5tHDArybZRnXaPS3pdkJ'
SECRET_ACCESS_KEY = 'lXXLM2U1dB3ExgKquGUwdGb88WHqkN'
ODPS_PROJECT = 'OpenDigger_prod_dev'
ODPS_ENDPOINT = 'http://service.cn-shanghai.maxcompute.aliyun.com/api'
# ACCESS_ID = 'LTAI5tSjrYp6JDW2bbiTdegU'
# SECRET_ACCESS_KEY = 'OE6JbSqOZUey5fzr9Wg6fuYYKvslZx'
# 创建odps实例
o = ODPS(ACCESS_ID, SECRET_ACCESS_KEY,
         project=ODPS_PROJECT, endpoint=ODPS_ENDPOINT)
options.tunnel.limit_instance_tunnel = False
# options.read_timeout = 3600000

In [3]:
users = DataFrame(o.get_table('ods_github_users')).to_pandas()

In [4]:
user_follow = users.loc[:,["database_id","followers","following"]]
user_follow_d = user_follow.drop_duplicates()

In [5]:
list_followers = list(map(lambda x: x.split(",") if x else None, [i for i in user_follow_d["followers"]]))
list_following = list(map(lambda x: x.split(",") if x else None, [i for i in user_follow_d["following"]]))
list_id = user_follow_d["database_id"].tolist()

In [6]:
G = nx.DiGraph()

In [7]:
for database_id, followers, following in user_follow_d.values:
    if followers:
        followers = followers.split(",")
        for follower in followers:
            G.add_edge(database_id, follower)
    if following:
        following = following.split(",")
        for one_following in following:
            G.add_edge(one_following, database_id)

In [8]:
# nx.draw(G, with_labels=False)

In [9]:
G.number_of_edges()

19069455

In [10]:
G.number_of_nodes()

3957111

In [11]:
pr = nx.pagerank(G)

In [19]:
sorted_pr = sorted(pr.items(), key= lambda item: item[1], reverse=True)

In [21]:
with open("./result/sorted_pagerank.txt", "w") as f:
    f.write("id\tpagerank\n")
    for item in sorted_pr:
        f.write(str(item[0])+"\t"+str(item[1])+"\n")

In [22]:
with open("./result/pagerank.txt", "w") as f:
    f.write("id\tpagerank\n")
    for key in pr:
        f.write(str(key)+"\t"+str(pr[key])+"\n")

In [14]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

h, a = nx.hits(G)

In [23]:
with open("./result/hits_h.txt", "w") as f:
    f.write("id\th\n")
    for key in h:
        f.write(str(key)+"\t"+str(h[key])+"\n")

In [24]:
with open("./result/hits_a.txt", "w") as f:
    f.write("id\ta\n")
    for key in a:
        f.write(str(key)+"\t"+str(a[key])+"\n")